# Data aquisition 

We used the **PRAW** (Python Reddit API Wrapper) library to programmatically access Reddit data through its API.


In [ ]:
!pip install praw pandas numpy

Importing the required packages:

In [17]:
import praw
import pandas as pd

Setting up credentials required to authenticate with Reddit's API using PRAW:

In [ ]:
# --- CONFIG ---
CLIENT_ID = '3Ptv1n3uzKL-RaqAQnrMlg'
CLIENT_SECRET = 'pa5OheU7NtiIw6jl5MaFAz8ouLrZDQ'
USER_AGENT = 'reddit-popularity-predictor'

Fetching posts from a range of different subbreddits:

In [ ]:
SUBREDDITS = ['technology', 'sports', 'funny', 'science', 'politics', 'gaming', 'movies']
POSTS_PER_SUBREDDIT = 750
SAMPLE_PER_BUCKET = 300 # how many posts per popularity bucket to keep

In [15]:
# Initialize Reddit API
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)

In [16]:

def fetch_posts(subreddit, sort, limit):
    """Fetch posts from a subreddit with given sort and limit."""
    posts = []
    submissions = getattr(reddit.subreddit(subreddit), sort)(limit=limit)
    for submission in submissions:
        posts.append({
            'subreddit': subreddit,
            'id': submission.id,
            'title': submission.title,
            'selftext': submission.selftext,
            'score': submission.score,
            'num_comments': submission.num_comments,
            'created_utc': submission.created_utc,
            'flair': submission.link_flair_text,
            'upvote_ratio': submission.upvote_ratio,
            'is_self': submission.is_self,
            'nsfw': submission.over_18,
            'author': str(submission.author),
            'url': submission.url,
            'sort_type': sort
        })
    return posts


In [ ]:
all_posts = []

# Fetch different types of posts (new posts, top posts)
for sub in SUBREDDITS:
    print(f"Fetching new posts from r/{sub}...")
    all_posts.extend(fetch_posts(sub, 'new', POSTS_PER_SUBREDDIT))

for sub in SUBREDDITS:
    print(f"Fetching top posts from r/{sub}...")
    all_posts.extend(fetch_posts(sub, 'top', POSTS_PER_SUBREDDIT))

# Create DataFrame
df = pd.DataFrame(all_posts)

# Remove duplicates (some posts may appear in both new and top)
df = df.drop_duplicates(subset='id')

print(f"Total posts before bucketing: {len(df)}")

Fetching new posts from r/technology...
Fetching new posts from r/sports...
Fetching new posts from r/funny...
Fetching new posts from r/science...
Fetching new posts from r/politics...
Fetching new posts from r/gaming...
Fetching new posts from r/movies...
Fetching top posts from r/technology...
Fetching top posts from r/sports...
Fetching top posts from r/funny...
Fetching top posts from r/science...
Fetching top posts from r/politics...
Fetching top posts from r/gaming...
Fetching top posts from r/movies...
Total posts before bucketing: 10095


In [7]:
df.head()

,subreddit,id,title,selftext,score,num_comments,created_utc,flair,upvote_ratio,is_self,nsfw,author,url,sort_type
0,technology,1ly9hhz,Generative AI is Turning Publishing Into a Swa...,,0,0,1.752350e+09,Artificial Intelligence,0.50,False,False,ubcstaffer123,https://www.pastemagazine.com/books/publishing...,new
1,technology,1ly9dpp,Can AI help prevent homelessness?,,0,5,1.752350e+09,Society,0.40,False,False,ubcstaffer123,https://www.thedesertreview.com/news/state/can...,new
2,technology,1ly851k,Why forecasters are concerned about losing 3 k...,,30,6,1.752347e+09,Society,0.94,False,False,Wagamaga,https://www.pbs.org/newshour/science/why-forec...,new
3,technology,1ly7o4a,Is AI Taking Over Influencers' Jobs?,,0,22,1.752345e+09,Artificial Intelligence,0.33,False,False,ThickTrack1492,https://www.aiviralclub.com/is-ai-taking-over-...,new
4,technology,1ly6fe5,The Trump Administration Is Planning to Use AI...,,3369,237,1.752342e+09,Artificial Intelligence,0.98,False,False,Aggravating_Money992,https://truthout.org/articles/the-trump-admini...,new


Instead of predicting Reddit post scores (a regression task), we simplify the problem into a classificatiion task by categorizing the scores into buckets (low, medium, high popularity)

The post scores are divided into the three categories based on quantiles. This helps to transform the continuous `score` into a new categorical variable, `popularity_bucket` which can be useful for classification models.


In [8]:
# --- Bucket scores into low/medium/high popularity ---

# Define buckets by score quantiles or fixed thresholds
# Here: Use quantiles to split into 3 equal groups

quantiles = df['score'].quantile([0.33, 0.66]).values
low_threshold, high_threshold = quantiles[0], quantiles[1]

def bucket_score(score):
    if score <= low_threshold:
        return 'low'
    elif score <= high_threshold:
        return 'medium'
    else:
        return 'high'

df['popularity_bucket'] = df['score'].apply(bucket_score)

print(df['popularity_bucket'].value_counts())

popularity_bucket
high      3432
low       3333
medium    3330
Name: count, dtype: int64


Since the dataset is already balanced across the `popularity_bucket` categories, we don#t need to apply additional sampling techniques to balance the data.

In [9]:
print("Length of the dataset:", len(df))

Length of the dataset: 10095


Finally, we save the data to a csv file:

In [11]:
# --- Save dataset ---
df.to_csv('reddit_dataset.csv', index=False)
print("Saved dataset to reddit_dataset.csv")

Saved dataset to reddit_dataset.csv
